# SQL (Python)

Resource: https://docs.python.org (Search sqlite3)

## sqlite3

Resource: https://docs.python.org/3/library/sqlite3.html

In [ ]:
import sqlite3

In [ ]:
# Create a database and connnection (con)
con = sqlite3.connect("tutorial.db")

In [ ]:
# Database cursor (cur)
cur = con.cursor()

In [ ]:
cur.execute("CREATE TABLE movie(title, year, score)")

In [ ]:
# Result
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

In [ ]:
# None
# Result
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
res.fetchone() is None

In [ ]:
# Add two rows
cur.execute("""
    INSERT INTO movie VALUES
        ('MOVIE 1', 2023, 9),
        ('MOVIE 2', 2023, 9.5)
""")

In [ ]:
# Commit the transaction
con.commit()

In [ ]:
# Result
res = cur.execute("SELECT score FROM movie")
res.fetchone()

In [ ]:
# Insert more rows
data = [
    ("Movie 4", 2022, 8),
    ("Movie 5", 2022, 8.5),
    ("Movie 6", 2022, 8.7)
]

In [ ]:
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()

In [ ]:
# Veriy new rows
for row in cur.execute("""
                        SELECT year, title
                        FROM movie
                        ORDER By year
                        """):
    print(row)         

In [ ]:
# CLose the connection
con.close()

In [ ]:
# Opening a new one
new_con = sqlite3.connect("tutorial.db")
new_cur = new_con.cursor()

In [ ]:
res = new_cur.execute("""
                        SELECT title, year
                        FROM movie
                        ORDER By score DESC""")

In [ ]:
title, year = res.fetchone()

In [ ]:
print(f'The highest score is {title!r}, released in {year}')

## Notepad (sqlite3 and streamlit)

In [ ]:
# Install streamlit
!pip install streamlit -q

In [ ]:
# Import libraries
import sqlite3
import streamlit as st

In [ ]:
# Create a folder
import os
dir_name = "app"
if dir_name in os.listdir():
    print("Directory is already created.")
else:
    os.mkdir(dir_name)
    print("Successfully created.")

In [ ]:
# Change directory
os.chdir(dir_name)

In [ ]:
# Print working directory
os.getcwd()

In [ ]:
%%writefile app.py
# Import libraries
import streamlit as st
import sqlite3
import os

# Header
st.subheader("Simple Notepad :notebook:")
st.caption("""
            Add your thoughts here! It will be stored in a database! \n
            :warning: :red[Do not add sensitive data].
            """)
st.subheader("",divider="rainbow")

# Variable
database_name = "notes.db"
con = sqlite3.connect(database_name)
cur = con.cursor()

# Create a database
if database_name in os.listdir():
    pass
else:
    con = sqlite3.connect(database_name)
    # Database Cursor (cur)
    cur = con.cursor()
    cur.execute("CREATE TABLE notes(name, note)")

# Inputs
name = st.text_input("Your Name here")
note = st.text_area("Add Note here")
if st.button("Add a note"):
    st.write(name,":", note)
    st.success("Successful Added.")
    # st.balloons()
    ### Insert into adatabase
    cur.execute(f"""
            INSERT INTO notes VALUES
            ("{name}", "{note}")
            """)
    con.commit()
    
# Previous Notes 
st.subheader("",divider="rainbow")
st.write("**Previous Notes**")
# Write the data
result = cur.execute("""
                    SELECT * 
                    FROM notes
                    ORDER BY name DESC
                    """)
for name, note in result.fetchall():
    st.write(name,":", note)
    
# Close Connection
con.close()

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit -q
COPY app.py .
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

In [ ]:
# Build an image
!docker build -t simple-notepad-app . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 8501:8501 -v $(pwd):/app simple-notepad-app

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

In [ ]:
# Create a firewall (GCP)
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:8501,tcp:8000,tcp:9000 --source-ranges=0.0.0.0/0